In [1]:
%env MINERL_DATA_ROOT=/Volumes/CORSAIR/data

env: MINERL_DATA_ROOT=/Volumes/CORSAIR/data


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import minerl  # NOTE: we need gym>=0.13.1,<0.20
import gym
from train import get_agent, get_dynamics_environment, FMC, get_data_handler
from fgz.training.fgz_trainer import FGZTrainer
from fgz_config import FGZConfig
import torch
import wandb
from tqdm import tqdm

/Users/dyllan/miniconda3/envs/minerl/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
torch.cuda.empty_cache()  # fix memory leaks

In [5]:
# enabled_tasks = [2, 3]  # cave and waterfall 
enabled_tasks = [0, 1, 2, 3]  # all 

config = FGZConfig(
    enabled_tasks=enabled_tasks,
    disable_fmc_detection=True,  # if true, only classification will occur. 
    use_wandb=True,
    unroll_steps=64,
)

In [6]:
minerl_env = gym.make('MineRLBasaltMakeWaterfall-v0')
agent = get_agent(config)
dynamics_env = get_dynamics_environment(config)

Loading model foundation-model-1x.model
with weights foundation-model-1x.weights


In [7]:
agent.device

device(type='cpu')

In [8]:
# dynamics_env.batched_action_space_sample()

In [9]:
# dummy_initial_state = torch.ones(4096, dtype=float)
# dynamics_env.set_all_states(dummy_initial_state)

In [10]:
data_handler = get_data_handler(config, agent)
for loader in data_handler.loaders:
    print(loader)

ContiguousTrajectoryDataLoader(n=35, /Volumes/CORSAIR/data/MineRLBasaltBuildVillageHouse-v0)
ContiguousTrajectoryDataLoader(n=30, /Volumes/CORSAIR/data/MineRLBasaltCreateVillageAnimalPen-v0)
ContiguousTrajectoryDataLoader(n=30, /Volumes/CORSAIR/data/MineRLBasaltFindCave-v0)
ContiguousTrajectoryDataLoader(n=30, /Volumes/CORSAIR/data/MineRLBasaltMakeWaterfall-v0)


In [11]:
# t = data_handler.sample_single_trajectory()
# for window in t:
#     print(len(window))
#     for frame, state_embedding, action in window:
#         print(state_embedding.shape)
#         break
#     break

In [12]:
dynamics_function_optimizer = torch.optim.Adam(
    dynamics_env.dynamics_function.parameters(),
    lr=0.001,
    # weight_decay=1e-4,
)

lr_scheduler = None
# lr_scheduler = torch.optim.lr_scheduler.StepLR(dynamics_function_optimizer, step_size=10, gamma=0.95)

In [13]:
fmc = FMC(dynamics_env, freeze_best=True)
trainer = FGZTrainer(agent, fmc, data_handler, dynamics_function_optimizer, config=config)

if config.use_wandb:
    wandb.init(project="fgz-v0.1.1", config=config.asdict())

2022-10-18 14:58:30 dyllans-MacBook-Pro.local wandb.jupyter[10557] ERROR Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
2022-10-18 14:58:32 dyllans-MacBook-Pro.local urllib3.connectionpool[10557] DEBUG Starting new HTTPS connection (1): api.wandb.ai:443
2022-10-18 14:58:32 dyllans-MacBook-Pro.local urllib3.connectionpool[10557] DEBUG https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
2022-10-18 14:58:32 dyllans-MacBook-Pro.local urllib3.connectionpool[10557] DEBUG Starting new HTTPS connection (1): api.wandb.ai:443
2022-10-18 14:58:32 dyllans-MacBook-Pro.local urllib3.connectionpool[10557] DEBUG https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
wandb: Currently logged in as: dyllan. Use `wandb login --relogin` to force relogin
2022-10-18 14:58:32 dyllans-MacBook-Pro.local git.cmd[10557] DEBUG Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/dyllan/Code/basalt_2022_compet

In [14]:
# trainer.save("test_trainer_save.pth")
# loaded_trainer = FGZTrainer.load("test_trainer_save.pth", agent)
# loaded_trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)

In [15]:
# trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)

In [16]:
# trainer.eval_actions

In [17]:
train_steps = 100
batch_size = 16
# train_steps = 5
checkpoint_every = 10
for train_step in tqdm(range(train_steps), desc="Training"):
    trainer.train_sub_trajectories(batch_size=batch_size, use_tqdm=False)
    if train_step % checkpoint_every == 0:
        trainer.save("fgz_dynamics_checkpoint.pth")

    if lr_scheduler is not None:
        lr_scheduler.step()

Training:   0%|          | 0/100 [00:00<?, ?it/s]/Users/dyllan/miniconda3/envs/minerl/lib/python3.7/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
2022-10-18 14:59:34 dyllans-MacBook-Pro.local urllib3.connectionpool[10557] DEBUG Starting new HTTPS connection (1): o151352.ingest.sentry.io:443
2022-10-18 14:59:34 dyllans-MacBook-Pro.local urllib3.connectionpool[10557] DEBUG https://o151352.ingest.sentry.io:443 "POST /api/5288891/envelope/ HTTP/1.1" 200 2


In [ ]:
# trainer.save("fgz_dynamics_trained.pth")

In [ ]:
# trainer = FGZTrainer.load("fgz_dynamics_trained.pth", agent)

In [ ]:
trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=256, force_no_escape=True)